In [1]:
{-# LANGUAGE NegativeLiterals #-}
{-# LANGUAGE FlexibleContexts #-}
{-# LANGUAGE OverloadedStrings #-}
{-# LANGUAGE TypeFamilies #-}
{-# LANGUAGE BangPatterns #-}

In [2]:
import Prelude hiding (Left, Right)
import Data.List
import qualified Data.Map as M

In [3]:
type Point = (Int, Int)

data Flag = Clean | Weakened | Infected | Flagged deriving (Show, Eq)

type Infection = M.Map Point Flag

data Direction = Up | Right | Down | Left deriving (Show, Eq, Enum)

data World = World { infected :: Infection
                   , position :: Point
                   , direction :: Direction
                   , infectionCount :: Int
                   } deriving (Eq, Show)
                   

In [4]:
text <- readFile "../../data/advent22.txt"
grid = lines text

In [5]:
sampleGrid = lines "..#\n#..\n...\n"

In [7]:
initialInfected g = M.fromList [((r, c), Infected) | r <- [0..(length g - 1)], c <- [0..((length . head) g - 1)],
                                        g!!r!!c == '#']

In [8]:
initialInfected sampleGrid

fromList [((0,2),Infected),((1,0),Infected)]

In [9]:
initialPosition g = (length g `div` 2, (length . head) g `div` 2)

In [10]:
initialPosition sampleGrid

(1,1)

In [11]:
leftOf Up = Left
leftOf x = pred x

rightOf Left = Up
rightOf x = succ x

In [12]:
leftOf Left

Down

In [13]:
delta :: Direction -> Point
delta Up = (-1, 0)
delta Right = (0, 1)
delta Down = (1, 0)
delta Left = (0, -1)

In [14]:
(+:) (r, c) (dr, dc) = (r + dr, c + dc)

In [15]:
initialWorld grid = World 
    { infected = initialInfected grid
    , position = initialPosition grid
    , direction = Up
    , infectionCount = 0
    }

In [16]:
initialWorld sampleGrid

World {infected = fromList [((0,2),Infected),((1,0),Infected)], position = (1,1), direction = Up, infectionCount = 0}

In [17]:
step world = World {infected = inf', position = pos', direction = dir', infectionCount = ic'}
    where here = position world
          stateHere = M.findWithDefault Clean here (infected world)
          dir' = case stateHere of 
                      Clean -> leftOf (direction world)
                      Weakened -> direction world
                      Infected -> rightOf (direction world)
                      Flagged -> rightOf (rightOf (direction world))
          stateHere' = case stateHere of 
                      Clean -> Weakened
                      Weakened -> Infected
                      Infected -> Flagged
                      Flagged -> Clean
          inf' = M.insert here stateHere' (infected world)
          
          ic'  = if stateHere' == Infected then infectionCount world + 1
                                           else infectionCount world
          pos' = here +: delta dir'

In [19]:
step $ step $ initialWorld sampleGrid

World {infected = fromList [((0,2),Infected),((1,0),Flagged),((1,1),Weakened)], position = (0,0), direction = Up, infectionCount = 0}

In [21]:
progress n = (!! n) . iterate step 

In [25]:
progress 7 $ initialWorld sampleGrid

World {infected = fromList [((0,-1),Weakened),((0,0),Weakened),((0,2),Infected),((1,-1),Infected),((1,0),Clean),((1,1),Weakened)], position = (1,-2), direction = Left, infectionCount = 1}

In [26]:
infectionCount $ progress 100 $ initialWorld sampleGrid

26

In [27]:
infectionCount $ progress 10000000 $ initialWorld sampleGrid

2511944

In [28]:
infectionCount $ progress 10000000 $ initialWorld grid

2512008